In [320]:
# default_exp evaluateSearch

# Evaluate module

> Wrapper for OpenAI to evaluate f1 performance of searching for terms in a dataset. Requires labeled dataframe.

In [321]:
#hide
from nbdev.showdoc import *

In [322]:
#export
import openai, pandas as pd, numpy as np, datetime, json, time, os
from OpenAISurveyWrapper import wrapper

In [464]:
#export
class evaluater(wrapper.wrapper):
    """
    wrapper on the API to load in survey definitions
    a format for displaying
    
    self.myFormat: a function to format a prompt
    """

    labeled = pd.DataFrame()
    raw = pd.DataFrame()
    examples = pd.DataFrame()
    searchTerms = {}
                        
    def addSearchTerm(self, label:str, searchTerm:str):
        """
        k, this is badly named, but I want to be able to evaluate how search does
        """
        try:
            self.searchTerms[label]["searchTerm"] = searchTerm
        except:
            self.searchTerms[label]= {"searchTerm":searchTerm}
    
    def removeSearchTerm(self, term:str):
        """
        delete search term
        """
        del self.searchTerms[term]
        
    def loadExamplesFromDataFrame(self, examples:dict):
        """
        takes a 2 column dataframe; first column should be "text", second "label"
        """
        if len(examples.columns)!=2:
            raise Exception("need 2 columns, first should be data second label")
        self.examples = examples
        self.examples.columns=["text", "label"]

            
    def printSearchTerms(self):
        """
        print off what I'm going to be searching
        """
        for term in self.searchTerms:
            print("{}: {}".format(term, self.searchTerms[term]["searchTerm"]))
                
    def optimizeSingleSearchCutoff(self, df:pd.DataFrame, term:str, outputDir:str, trainSize=200, testSize=200, sleepTime:float=2):
        """
        run a search against the df texts to get most similar
        the API returns the same similarity scale when the same term is queried multiple times against different docs

        saves the output by the name of the search query to the outputDir
        """
        
        if df.columns[0] != "text" or df.columns[1] !="label":
            print("pass a df with columns label and text")
            print(df.columns)
            return
        
        responses = []
        trgts = df[:trainSize + testSize].copy()
        startIdx = 0
        endIdx = 200
        while startIdx < len(trgts):
            #responses = responses + trgts[startIdx:endIdx].text.to_list()

            responses = responses + openai.Engine("davinci").search(documents=\
                        trgts[startIdx:endIdx].text.to_list(), query=term)["data"]
            startIdx += 200
            endIdx += 200
            time.sleep(sleepTime)

        trgts["seach_sim"] = [float(x["score"]) for x in responses]

        cutoff_f1 = pd.DataFrame()
        for i in range(0, 700):
            trgts["search_pos"] = trgts["seach_sim"].apply(lambda x: 1 if x > i else 0)
            cutoff_f1.at["f1", i] = f1_score(trgts.label[:trainSize], trgts.search_pos[:trainSize])

        cutoff = cutoff_f1.T.idxmax()[0]

        cutoff_f1_test = pd.DataFrame()
        for i in range(0, 700):
            trgts["search_pos"] = trgts["seach_sim"].apply(lambda x: 1 if x > i else 0)
            cutoff_f1_test.at["f1", i] = f1_score(trgts.label[trainSize:], trgts.search_pos[trainSize:])

        if outputDir != "":
            os.makedirs(outputDir, exist_ok=True)
            trgts.to_pickle("{}/{}.pkl".format(outputDir, term))

        return cutoff_f1_test[cutoff][0]
    
    def generateSearchScoresForLabel(self, outFileName="", trainSize=200, testSize=200):
        """
        autogeneratres the search scores
        """
        
        results = pd.DataFrame()
        for term in self.searchTerms:
            search_term = self.searchTerms[term]["searchTerm"]
            print(search_term)
            myResult = self.optimizeSingleSearchCutoff(self.examples, search_term, outFileName, trainSize=trainSize, testSize=testSize)
            print(myResult)
            results.at[search_term, "testF1"] = myResult

        self.results = results    
        return results

In [484]:
tmp = evaluater()

In [485]:
tmp.add_key(json.load(open("../gpt3/key.json", "r"))["key"])

In [474]:
#hide
df = pd.read_csv("../vice-yikyak-2020-06-09_vice-random-2020-06-23.csv")

In [475]:
df.head()

,Drinking,Tobacco,Vaping,Marijuana,Illegal drugs,Prescription drugs,text
0,0,0,0,0,0,0,I'm 20 years old and still can't figure out where my clit is 😒😞
1,0,0,0,0,0,0,A beard can either make or break you
2,0,0,0,0,0,0,"You know what would get me hard right now, you whippin your dick out! 👬"
3,0,0,0,0,0,0,When did the bitch from frozen come to auburn? This is getting a little ridiculous
4,0,0,0,0,0,0,"Do guys find tall girls attractive, like +6foot?"


In [478]:
df[df.columns[:5]].sum()

Drinking         402
Tobacco          23 
Vaping           6  
Marijuana        203
Illegal drugs    52 
dtype: int64

In [479]:
#hide
tmp.loadExamplesFromDataFrame(df[["text", "Marijuana"]])

In [480]:
len(df)

9999

In [481]:
tmp.addSearchTerm("1", "Marijuana")

In [482]:
tmp.printSearchTerms()

1: Marijuana


In [483]:
tmp.generateSearchScoresForLabel(outFileName="testing")

Marijuana
0.4


,testF1
Marijuana,0.4
